# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for A. Thaliana

# Imports

In [1]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE, VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator,TransformerMixin

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [2]:
ORGANISM = "ecoli"
TAX_ID = 83333
LOG_FILE = f"../logs/{ORGANISM}_amino_sugar.log"
N_THREADS = 16
IDENTITY_THRESHOLD=70
OUTLIERS = [
    "P0AAG8",
    "P04983",
    "P10346",
    "P28635",
    "P14175",
    "P69797",
    "P76773",
    "P37388",
    "P30750",
    "Q47706",
    "P10907",
    "P37774",
    "P0AAF3",
    "P64550",
    "P02943",
    "P75733",
    "P0AAF6",
    "P68187",
    "P69856",
    "P07109",
    "Q6BEX0",
]



# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    outliers=OUTLIERS,
    verbose=True,
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P69801,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,manY pel ptsP b1818 JW1807,PTS system mannose-specific EIIC component (EI...,4.A.6.1.1,83333,MEITTLQIVLVFIVACIAGMGSILDEFQFHRPLIACTLVGIVLGDM...
P36672,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,treB b4240 JW4199,PTS system trehalose-specific EIIBC component ...,4.A.1.2.4,83333,MMSKINQTDIDRLIELVGGRGNIATVSHCITRLRFVLNQPANARPK...
P56580,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,srlE gutA gutE b2703 JW5430,PTS system glucitol/sorbitol-specific EIIB com...,4.A.4.1.1,83333,MTHIRIEKGTGGWGGPLELKATPGKKIVYITAGTRPAIVDKLAQLT...
P0AA47,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,plaP yeeF b2014 JW5330,Low-affinity putrescine importer PlaP,2.A.3.1.14,83333,MSHNVTPNTSRVELRKTLTLVPVVMMGLAYMQPMTLFDTFGIVSGL...
P08722,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,bglF bglC bglS b3722 JW3700,PTS system beta-glucoside-specific EIIBCA comp...,4.A.1.2.2,83333,MTELARKIVAGVGGADNIVSLMHCATRLRFKLKDESKAQAEVLKKT...
...,...,...,...,...,...,...,...,...
P19642,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,malX b1621 JW1613,PTS system maltose-specific EIICB component [I...,4.A.1.1.3,83333,MTAKTAPKVTLWEFFQQLGKTFMLPVALLSFCGIMLGIGSSLSSHD...
P0AAD4,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,tyrP b1907 JW1895,Tyrosine-specific transport protein (Tyrosine ...,2.A.42.1.1,83333,MKNRTLGSVFIVAGTTIGAGMLAMPLAAAGVGFSVTLILLIGLWAL...
P23173,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,tnaB trpP b3709 JW5619/JW5622,Low affinity tryptophan permease,2.A.42.1.3,83333,MTDQAEKKHSAFWGVMVIAGTVIGGGMFALPVDLAGAWFFWGAFIL...


## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P69801,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,manY pel ptsP b1818 JW1807,PTS system mannose-specific EIIC component (EI...,4.A.6.1.1,83333,MEITTLQIVLVFIVACIAGMGSILDEFQFHRPLIACTLVGIVLGDM...
P36672,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,treB b4240 JW4199,PTS system trehalose-specific EIIBC component ...,4.A.1.2.4,83333,MMSKINQTDIDRLIELVGGRGNIATVSHCITRLRFVLNQPANARPK...
P56580,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,srlE gutA gutE b2703 JW5430,PTS system glucitol/sorbitol-specific EIIB com...,4.A.4.1.1,83333,MTHIRIEKGTGGWGGPLELKATPGKKIVYITAGTRPAIVDKLAQLT...
P0AA47,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,plaP yeeF b2014 JW5330,Low-affinity putrescine importer PlaP,2.A.3.1.14,83333,MSHNVTPNTSRVELRKTLTLVPVVMMGLAYMQPMTLFDTFGIVSGL...
P08722,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,bglF bglC bglS b3722 JW3700,PTS system beta-glucoside-specific EIIBCA comp...,4.A.1.2.2,83333,MTELARKIVAGVGGADNIVSLMHCATRLRFKLKDESKAQAEVLKKT...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
).labels.value_counts()

Amino-acid transport    51
Sugar transport         47
Name: labels, dtype: int64

## PSSM

In [7]:
for uniref_cluster_threshold in [50, 90]:
    for psiblast_iterations in [1, 3]:
        calculate_pssm_feature(
            input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
            output_tsv="../data/features/{}_amino_sugar_cluster{}_pssm_ur{}_{}it.tsv".format(
                ORGANISM,
                IDENTITY_THRESHOLD,
                uniref_cluster_threshold,
                psiblast_iterations,
            ),
            tmp_folder="../data/intermediate/blast/pssm_uniref{}_{}it".format(
                uniref_cluster_threshold, psiblast_iterations
            ),
            blast_db="../data/raw/uniref/uniref{}/uniref{}.fasta".format(
                uniref_cluster_threshold, uniref_cluster_threshold
            ),
            iterations=psiblast_iterations,
            psiblast_executable="psiblast",
            psiblast_threads=N_THREADS,
            verbose=False,
        )


## Reading dataframes

In [8]:
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_pssm_50_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_1it.tsv",
    index_col=0,
)
df_pssm_50_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_3it.tsv",
    index_col=0,
)
df_pssm_90_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_1it.tsv",
    index_col=0,
)
df_pssm_90_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_3it.tsv",
    index_col=0,
)

## Combining dataframes

In [9]:
df_pssm_all = pd.concat(
    [
        df_pssm_50_1it.rename(columns=lambda c: c + "_50_1"),
        df_pssm_50_3it.rename(columns=lambda c: c + "_50_3"),
        df_pssm_90_1it.rename(columns=lambda c: c + "_90_1"),
        df_pssm_90_3it.rename(columns=lambda c: c + "_90_3"),
    ],
    axis=1,
)


## Custom Transformer to try all parameters

In [10]:
class PSSMSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, uniref_threshold="all", iterations="all"):
        self.feature_names = feature_names
        self.uniref_threshold = uniref_threshold
        self.iterations = iterations

    def fit(self, X, y=None):
        if self.uniref_threshold in {50, 90}:
            has_uniref = (
                np.char.find(self.feature_names, str(self.uniref_threshold)) >= 0
            )
        elif self.uniref_threshold == "all":
            has_uniref = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect uniref threshold {self.uniref_threshold}")

        if self.iterations in {1, 3}:
            has_iterations = np.char.find(self.feature_names, str(self.iterations)) >= 0
        elif self.iterations == "all":
            has_iterations = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect iteration count: {self.iterations}")
        self.mask = np.bitwise_and(has_uniref, has_iterations)
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        X = X[:, self.mask]
        return X


# Functions

In [11]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["sum"] = df_stats.sum(axis=1)
    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [12]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy(dtype=str)
    sample_names = df_features.index.to_numpy(dtype=str)
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [13]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Individual Features

## PSSM

### Stats, Plots

In [14]:
df_stats = get_feature_stats(df_pssm_50_1it, df_labels)


df_stats["shapiro_p"] = df_pssm_50_1it.apply(lambda col: shapiro(col)[1], axis=0).round(4)
df_stats["shapiro"] = df_pssm_50_1it.apply(lambda col: shapiro(col)[0], axis=0)

display(df_stats[df_stats.shapiro < 0.9])

df_stats.sort_values("std")

,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff,shapiro_p,shapiro
IL,-0.522114,0.913637,0.088292,0.479814,0.522114,0.957664,0.865862,1.000000,0.859060,0.140940,0.0,0.865145
QG,0.330220,0.954677,0.066043,1.350940,0.330220,0.933848,0.977279,0.947230,1.000000,0.052770,0.0,0.734246
IE,0.304483,0.079259,0.033101,0.416843,0.304483,0.069633,0.089704,0.067538,0.080537,0.012999,0.0,0.882909
ID,0.250907,0.008866,0.028263,0.288036,0.250907,0.002093,0.016215,0.000000,0.000000,0.000000,0.0,0.355416


,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff,shapiro_p,shapiro
ID,0.250907,0.008866,0.028263,0.288036,0.250907,0.002093,0.016215,0.000000,0.000000,0.000000,0.0000,0.355416
IE,0.304483,0.079259,0.033101,0.416843,0.304483,0.069633,0.089704,0.067538,0.080537,0.012999,0.0000,0.882909
IK,0.246378,0.123921,0.034493,0.404793,0.246378,0.115805,0.132729,0.112805,0.127946,0.015141,0.0000,0.913742
IQ,0.156013,0.193430,0.036534,0.385976,0.156013,0.187986,0.199337,0.186842,0.200557,0.013715,0.4321,0.986694
IN,0.213718,0.105122,0.038240,0.357079,0.213718,0.097316,0.113591,0.100796,0.103704,0.002908,0.1856,0.981560
...,...,...,...,...,...,...,...,...,...,...,...,...
QF,-0.357977,0.283980,0.112479,0.038482,0.357977,0.322436,0.242251,0.336788,0.231373,0.105415,0.0489,0.973996
QL,-0.433425,0.252155,0.113168,-0.068103,0.433425,0.299001,0.201322,0.320442,0.197772,0.122670,0.0356,0.972197
GD,-0.229947,0.189484,0.114482,0.074019,0.229947,0.214627,0.162203,0.216216,0.170732,0.045485,0.0048,0.960323
QI,-0.384288,0.266122,0.114760,-0.003406,0.384288,0.308242,0.220418,0.335526,0.212042,0.123484,0.0124,0.966076


### Independent test set

In [15]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_pssm_all, df_labels, test_size=0.2)

### Model selection

SVC (with default RBF kernel) looks the most promising.

In [16]:
for estimator in [
    LinearSVC(max_iter=1e6, class_weight="balanced", random_state=0),
    SVC(class_weight="balanced"),
    RandomForestClassifier(class_weight="balanced", random_state=0),
    LinearSVC(max_iter=1e6, random_state=0),
    SVC(),
    RandomForestClassifier(random_state=0),
    GaussianNB(),
    KNeighborsClassifier(),
    SGDClassifier(random_state=0),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(pipe, X_train, y_train, scoring="f1_macro")
    print("### ", str(estimator))
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC(class_weight='balanced', max_iter=1000000.0)
CV folds: [1.    0.937 0.937 0.661 0.932]
Mean: 0.893
Std: 0.119
###  SVC(class_weight='balanced')
CV folds: [0.619 0.746 0.812 0.722 0.661]
Mean: 0.712
Std: 0.067
###  RandomForestClassifier(class_weight='balanced')
CV folds: [0.676 0.619 0.875 0.722 0.661]
Mean: 0.711
Std: 0.089
###  LinearSVC(max_iter=1000000.0)
CV folds: [1.    0.937 0.937 0.661 0.932]
Mean: 0.893
Std: 0.119
###  SVC()
CV folds: [0.792 0.746 0.875 0.722 0.661]
Mean: 0.759
Std: 0.072
###  RandomForestClassifier()
CV folds: [0.515 0.686 0.812 0.722 0.732]
Mean: 0.694
Std: 0.098
###  GaussianNB()
CV folds: [0.619 0.686 0.625 0.722 0.583]
Mean: 0.647
Std: 0.05
###  KNeighborsClassifier()
CV folds: [0.867 0.746 0.875 0.598 0.785]
Mean: 0.774
Std: 0.101
###  SGDClassifier()
CV folds: [0.935 0.806 0.873 0.598 0.732]
Mean: 0.789
Std: 0.117


### Parameter tuning

In [17]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        LinearSVC(max_iter=1e6, random_state=0),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [0.01, 0.1, 1],
        "linearsvc__dual": [True, False],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'linearsvc__C': 0.1, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 'all'}
0.9346031746031744


### Dimensionality reduction

In [18]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 97% of variance:", np.argmax(csum >= 0.97) + 1)

Number of components to explain 97% of variance: 40


In [19]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        LinearSVC(max_iter=1e6, random_state=0),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [0.001, 0.01, 0.1],
        "linearsvc__dual": [True, False],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc_pca = gsearch.best_estimator_

{'linearsvc__C': 0.001, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pca__n_components': 0.98, 'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 'all'}
0.9083143756827967


In [20]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [0.1, 1, 10],
        # "svc__gamma": [0.1, 0.01, 0.001,"scale"],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_pca = gsearch.best_estimator_

{'pca__n_components': 0.9299999999999999, 'pssmselector__iterations': 'all', 'pssmselector__uniref_threshold': 90, 'svc__C': 1, 'svc__class_weight': None}
0.908054298642534


In [21]:
# gsearch = GridSearchCV(
#     estimator=make_pipeline(
#         PSSMSelector(feature_names=feature_names),
#         StandardScaler(),
#         SelectKBest(),
#         StandardScaler(),
#         LinearSVC(max_iter=1e6),
#     ),
#     param_grid={
#         "pssmselector__uniref_threshold": [50, 90, "all"],
#         "pssmselector__iterations": [1, 3, "all"],
#         "linearsvc__class_weight": ["balanced"],
#         "linearsvc__C": [0.1, 1, 10],
#         "linearsvc__dual": [True, False],
#         "selectkbest__k": list(range(1,400))
#     },
#     cv=5,
#     scoring="f1_macro",
#     n_jobs=-1,
#     return_train_score=True,
# )
# gsearch.fit(X_train, y_train)
# print(gsearch.best_params_)
# print(gsearch.best_score_)
# best_estimator_lsvc_kbest = gsearch.best_estimator_

### Validation

In [22]:
best_estimator = best_estimator_svc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.908+-0.089


,Amino,Sugar,Macro,Weighted
precision,0.833,1.000,0.917,0.917
recall,1.000,0.800,0.900,0.900
f1-score,0.909,0.889,0.899,0.899
support,10.000,10.000,20.000,20.000


,Amino,Sugar
Amino,10,0
Sugar,2,8


In [23]:
best_estimator = best_estimator_lsvc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.908+-0.067


,Amino,Sugar,Macro,Weighted
precision,0.769,1.000,0.885,0.885
recall,1.000,0.700,0.850,0.850
f1-score,0.870,0.824,0.847,0.847
support,10.000,10.000,20.000,20.000


,Amino,Sugar
Amino,10,0
Sugar,3,7
